# Вычисление среднего вектора набора данных

Для представления текста в виде вектора используется бинарный вектор, для нормализации слов используется Yandex MyStem, 

# Модель новости

In [1]:
import json
import numpy as np
from pymystem3 import Mystem
import re
from sklearn.feature_extraction.text import CountVectorizer

class News:
    def __init__(self, id, date, annotation, title, content, url):
        self.id = id
        self.date = date
        self.annotation = annotation
        self.title = title
        self.content = content
        self.url = url
    
    @classmethod
    def from_json(cls, json_str):
        json_dict = json.loads(json_str)
        return cls(**json_dict)

# Загружаем новости

In [2]:
news = []
with open('/data/kasandra/news-1000.json', encoding="utf8") as f:
    for line in f:
        news.append(News.from_json(line))

list(map(lambda w: w.title, news[:5]))

['На Алтае заработала первая в РФ система контроля космического пространства',
 'ЛНР: Действия украинских диверсантов — одна из версий покушения на Плотницкого',
 'Bild не публикует результаты России в медальном зачёте Олимпиады',
 'Глава ЛНР связал покушение на свою жизнь с украинскими спецслужбами',
 'Мэра города в США арестовали за попытку расплатиться наркотиками за секс с мужчинами']

# Загружаем стоп слова

In [4]:
stopwords = []

with open('../res/stopwords.txt', mode="r", encoding="utf8") as file:
    for line in file:
        stopwords.append(line.replace('\n', ''))

# Подключаем Yandex MyStem для нормализации текста

In [5]:
mystem = Mystem(mystem_bin='/data/mystem/mystem',
                entire_input=False)

# Оставляем только русские слова
r = re.compile('^[А-ЯЙа-яй]*$')

def lemmatize(text):
    return list(filter(r.match, mystem.lemmatize(text)))

## Преобразуем текст в вектор с помощью TfidfVectorizer

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(use_idf=True,
                                   tokenizer=lemmatize,
                                   stop_words=stopwords,
                                   ngram_range=(1,3))

tfidf_matrix = tfidf_vectorizer.fit_transform(map(lambda n: n.content, news))
idf = tfidf_vectorizer.idf_

# Вычисление среднеарифметического вектора по всем данным

[Centroid](https://en.wikipedia.org/wiki/Centroid#Of_a_finite_set_of_points)

In [7]:
def mean_vector(bin_matrix):
    mean = bin_matrix[0].toarray()
    for v in bin_matrix[1:]:
        mean = mean + v.toarray()
    return mean / bin_matrix.shape[0]

In [7]:
mean = mean_vector(tfidf_matrix)
names = vectorizer.get_feature_names()
wd = zip(names, mean[0])

list(filter(lambda x: x[1] > 0.2, wd))

[('авг', 0.29499999999999998),
 ('важный', 0.30299999999999999),
 ('время', 0.26100000000000001),
 ('год', 0.42299999999999999),
 ('заявлять', 0.29299999999999998),
 ('кнопка', 0.28399999999999997),
 ('курс', 0.28699999999999998),
 ('любой', 0.33200000000000002),
 ('москва', 0.34300000000000003),
 ('мочь', 0.28899999999999998),
 ('нажимать', 0.28199999999999997),
 ('новость', 0.78300000000000003),
 ('отмечать', 0.22500000000000001),
 ('подписываться', 0.28100000000000003),
 ('проходить', 0.38100000000000001),
 ('ранее', 0.25600000000000001),
 ('российский', 0.30499999999999999),
 ('россия', 0.39700000000000002),
 ('самый', 0.39700000000000002),
 ('слово', 0.29799999999999999),
 ('сообщать', 0.39500000000000002),
 ('ссылка', 0.33800000000000002),
 ('становиться', 0.252),
 ('страна', 0.23200000000000001),
 ('устройство', 0.29299999999999998),
 ('человек', 0.30199999999999999)]

## Группировка новостей по дням

## Вычисление вектора для каждого дня

## Вычисляем средние вектора для каждого дня

# Вычисление отклонения для каждого дня

# Построение графика